In [36]:
import pandas as pd
import pickle
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader, Dataset
import tqdm
import torch.optim as optim
from itertools import islice
import math

In [37]:
## Accessing KG Node map dict
with open(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/tucson_KG_node_map_dict','rb')as f:
  KG_node_map_dict = pickle.load(f)

In [38]:
## Accessing Trained Secondary Model

# Define the model architecture
class CFNet(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(CFNet, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim, _weight=user_embeddings)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim, _weight=item_embeddings)
        self.fc1 = nn.Linear(embedding_dim * 2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embeddings(user_ids)
        item_embeds = self.item_embeddings(item_ids)
        x = torch.cat([user_embeds, item_embeds], dim=1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

with open(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/secondary_model_tucson', 'rb') as handle:
    secondary_model = pickle.load(handle)

In [39]:
## Accessing KG to secondary model map of users
with open(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/user_kg_index_secondary_id_map_tucson','rb') as f:
  user_kg_index_secondary_id_map = pickle.load(f)

## Accessing KG to secondary model map of items
with open(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/item_kg_index_secondary_id_map_tucson','rb') as f:
  item_kg_index_secondary_id_map = pickle.load(f)

In [40]:
## Function to get test POIs for a group
def get_test_poi_for_group(group):
  test_poi_group = []
  for user in group:
    for poi in userwise_test_poi[user]:
      test_poi_group.append(poi)

  return list(set(test_poi_group))

In [41]:
# Accessing heldout POIs for users
file = open(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/UP_dict_test', "rb")
UP_dict_test = pickle.load(file)

In [42]:
# Accessing user groups
file = open(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/tucson_group_8', "rb")
user_group = pickle.load(file)

In [43]:
## Creating userwise test POI dictonary
userwise_test_poi = {}
for user in UP_dict_test:
  userwise_test_poi[user] = []
  for record in UP_dict_test[user]:
    userwise_test_poi[user].append(record[0])

In [44]:
## Accessing Group Embeddings
with open(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/group8_embeddings_tucson','rb')as f:
  group_embeddings = pickle.load(f)

In [45]:
Visit_metadata = pd.read_csv(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/tucson_reviews.csv')

In [46]:
## Creating userwise visited POI rating dictonary
UP_dict = {}
for i in range(len(Visit_metadata)):
  user = Visit_metadata.iloc[i,1]
  if user not in UP_dict:
    UP_dict[user] = []
  poi = Visit_metadata.iloc[i,2]
  rating = Visit_metadata.iloc[i,3]
  UP_dict[user].append([poi,rating])

In [47]:
poi_metadata = pd.read_csv(r'/content/drive/MyDrive/Shopping POIs Final Data/Tucson City Data/Tucson City Data/tucson_poi_metadata.csv')

In [48]:
## Function to get test POIs for a group
def get_test_poi_for_group(group):
  test_poi_group = []
  for user in group:
    for poi in userwise_test_poi[user]:
      test_poi_group.append(poi)

  return list(set(test_poi_group))

In [49]:
## Function to get top K POI recommendation for group
def get_grp_recommendations(group_idx, candidate_grp_pois,K):
    group_embedding = group_embeddings[group_idx]
    # group_embedding = H1_grp_embedding
    predicted_ratings = []
    for poi in candidate_grp_pois:
        if poi in KG_node_map_dict and KG_node_map_dict[poi] in item_kg_index_secondary_id_map:
            item_id = item_kg_index_secondary_id_map[KG_node_map_dict[poi]]
            item_tensor = torch.LongTensor([item_id])
            item_embed = secondary_model.item_embeddings(item_tensor)[0]
            x = torch.cat([group_embedding, item_embed], dim=0)
            x = nn.functional.relu(secondary_model.fc1(x))
            x = secondary_model.fc2(x)
            predicted_ratings.append(float(x))
    pairs = list(zip(predicted_ratings, candidate_grp_pois))
    sorted_pairs = sorted(pairs, reverse=True)
    top_items = [pair[1] for pair in sorted_pairs[:K]]
    return top_items

In [50]:
recommenadtions_15_group_size_8 = {}
for i in range(len(user_group)):
  candidate_pois = get_test_poi_for_group(user_group[i])
  recommenadtions_15_group_size_8[i] = get_grp_recommendations(i,candidate_pois,15)


In [51]:
# Save using pickle
with open("/content/recommenadtions_15_group_size_8.pkl", "wb") as f:
    pickle.dump(recommenadtions_15_group_size_8, f)

In [52]:
import os

file_path = '/content/Dominant_Categories.pkl'
print(f"File size: {os.path.getsize(file_path)} bytes")


File size: 1152 bytes


In [53]:
import pickle

# Load the pickle file
file_path = '/content/Dominant_Categories.pkl'

with open(file_path, 'rb') as file:
    Dominant_Categories = pickle.load(file)


In [54]:
def get_filtered_categories_from_pois(poi_list, dominant_categories):
    # Normalize dominant categories
    dominant_set = set(cat.strip().lower() for cat in dominant_categories)
    poi_category_dict = {}

    for poi in poi_list:
        categories_str = poi_metadata.loc[poi_metadata['business_id'] == poi, 'categories']
        if not categories_str.empty:
            categories = [cat.strip() for cat in categories_str.iloc[0].split(',')]
            # Filter and normalize
            filtered = [cat for cat in categories if cat.strip().lower() in dominant_set]
            poi_category_dict[poi] = filtered

    return poi_category_dict


In [55]:
with open("/content/recommenadtions_15_group_size_8.pkl", "rb") as f:
    recommenadtions_15_group_size_8 = pickle.load(f)


In [56]:
dominant_categories_list_reco_grop_size_8_reco_15 = {}

In [57]:
for i in range(len(recommenadtions_15_group_size_8)):
  reco = recommenadtions_15_group_size_8[i]
  dominant_categories_list_reco_grop_size_8_reco_15[i] = get_filtered_categories_from_pois(reco,Dominant_Categories)

In [58]:
print(len(dominant_categories_list_reco_grop_size_8_reco_15[14]))

9


In [59]:
# Save using pickle
with open("/content/dominant_categories_list_reco_grop_size_8_reco_15", "wb") as f:
    pickle.dump(dominant_categories_list_reco_grop_size_8_reco_15, f)